# LLM Fine-Tuning Tutorial
## Complete Guide to Training and Deploying Your Model

This notebook walks you through the entire process.

## Step 1: Setup

In [ ]:
# Install required packages
!pip install -q torch transformers accelerate peft datasets tqdm

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## Step 2: Create Sample Data

In [ ]:
# Run the data generation script
!python ../scripts/create_sample_data.py --output_dir ../data/processed

# Verify data was created
!ls -lh ../data/processed/

## Step 3: Train the Model

In [ ]:
# Train with LoRA
!python ../training/train_lora.py \
    --model_name "TinyLlama/TinyLlama-1.1B-Chat-v1.0" \
    --dataset_path ../data/processed/train.jsonl \
    --output_dir ../models/tutorial_model \
    --num_epochs 1 \
    --batch_size 2 \
    --use_4bit

## Step 4: Test the Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load model
model_path = "../models/tutorial_model"
base_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    torch_dtype=torch.float16,
)
model = PeftModel.from_pretrained(base_model, model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Test inference
def generate_response(prompt, max_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Try it out
prompt = "What is machine learning?"
print(f"Question: {prompt}")
print(f"Answer: {generate_response(prompt)}")

## Step 5: Evaluate

In [ ]:
# Run evaluation
!python ../evaluation/evaluate.py \
    --model_path ../models/tutorial_model \
    --test_data ../data/processed/test.jsonl \
    --output_file ../results/eval_tutorial.json

## Next Steps

1. Try different models
2. Use your own data
3. Experiment with LoRA parameters
4. Deploy with the API server